In [1]:
import os, getpass
from os import getenv
from langchain_openai import AzureChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import ConfigurableFieldSpec
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [ ]:
if not getenv("AZURE_OPENAI_API_KEY"):
  os.environ["AZURE_OPENAI_API_KEY"] = getpass.getpass("Enter API key for Azure: ")
endpoint = getenv("AZURE_OPENAI_ENDPOINT")
# endpoint = getenv("AZURE_OPENAI_ENDPOINT").format(getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),  getenv("AZURE_OPENAI_API_VERSION"))

In [ ]:
# prompt = ChatPromptTemplate.from_template("Tell me a joke about {word} in 100 words.")
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful translator. Translate the user sentence to French.",
        ),
        ("human", "{input}"),
    ]
)
parser = StrOutputParser()
llm = AzureChatOpenAI(
    azure_endpoint=getenv("AZURE_OPENAI_ENDPOINT"),
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    temperature=0.01,
    stream_usage=True
)

chain = prompt | llm

In [82]:
messages = [  
(  
"system",  
"You are a helpful translator. Translate the user sentence to French.",  
),  
("human", "I love programming."),  
] 

In [22]:
await llm.ainvoke(messages)

AIMessage(content="J'adore la programmation.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 28, 'total_tokens': 34, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'id': 'chatcmpl-BaUDFPAFs2smbF11kYf2UkyY9cNe0', 'service_tier': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_

In [ ]:
# get the full response at once
if chain:
    response = await chain.ainvoke({"input": "story on earth"})
    print(response)

content='histoire sur Terre' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 27, 'total_tokens': 32, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0', 'id': 'chatcmpl-BaV2XvCohIQJaJ53D0Calgk8GJmiu', 'service_tier': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'f

In [ ]:
# stream the response
full_message = None

async for chunk in chain.astream({"input": "on earth"}):
    # print(chunk.content, end="", flush=True)
    if full_message is None:
        full_message = chunk
    else:
        full_message += chunk  # Uses the overloaded __add__ method

# After streaming is complete, full_message contains the aggregated content and metadata
print(full_message)


content="J'adore la programmation." additional_kwargs={} response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0'} id='run--0c4d065d-a4d6-4abb-82de-e751b4a7be11' usage_metadata={'input_tokens': 28, 'output_tokens': 6, 'total_tokens': 34, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [84]:
print(type(full_message))

<class 'langchain_core.messages.ai.AIMessageChunk'>


**Chat history in memory with proxy history upto last 2 messages**

In [ ]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage
from pydantic import BaseModel, Field

class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    messages: list[BaseMessage] = Field(default_factory=list)

    def add_messages(self, messages: list[BaseMessage]) -> None:
        self.messages.extend(messages)

    def clear(self) -> None:
        self.messages = []

class ProxyHistory(BaseChatMessageHistory):
    def __init__(self, full_history: InMemoryHistory, limit: int = 2):
        self._full_history = full_history
        self._limit = limit

    @property
    def messages(self) -> list[BaseMessage]:
        # Only return the last N messages
        return self._full_history.messages[-self._limit:]

    def add_messages(self, messages: list[BaseMessage]) -> None:
        # Delegate writes to the original history
        self._full_history.add_messages(messages)

    def clear(self) -> None:
        self._full_history.clear()


store = {}
session_id = "student1234"

def get_by_session_id(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryHistory()
    return ProxyHistory(store[session_id], limit=2)



In [ ]:
# prompt = ChatPromptTemplate.from_template("Tell me a joke about {word} in 100 words.")
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}"),
])
parser = StrOutputParser()
llm = AzureChatOpenAI(
    azure_endpoint=endpoint,
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    temperature=0.01,
    streaming=True,
)

chain = prompt | llm | parser
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history=get_by_session_id,
    input_messages_key="input",
    history_messages_key="history",
)

In [ ]:
async for chunk in chain_with_history.astream({"input": "how many messages we have in this chat? can you list them with roles?"},
    config={"configurable": {"session_id": session_id}}):
    print(chunk, end="", flush=True)

In [ ]:
print(store)

In [ ]:
import pickle
serialized_store = pickle.dumps(store)
print(serialized_store)

In [ ]:
re_store = pickle.loads(serialized_store)
store=re_store
print(re_store)

**Chat history with Scylla DB**

In [ ]:
# Create keyspace and tables
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider

# Connect to ScyllaDB
cluster = Cluster(['127.0.0.1'])  # Replace with your Scylla host IP if different
session = cluster.connect()

# Create keyspace (if not exists)
KEYSPACE = "eva"
session.execute(f"""
    CREATE KEYSPACE IF NOT EXISTS {KEYSPACE}
    WITH replication = {{'class': 'SimpleStrategy', 'replication_factor': '1'}}
""")

# Set keyspace
session.set_keyspace(KEYSPACE)

# Create tables
table_queries = [
    """
    CREATE TABLE IF NOT EXISTS chathistory (
        userid uuid,
        chatid uuid,
        visible boolean,
        chathistoryjson blob,
        chattitle text,
        createdon timestamp,
        nettokenconsumption int,
        PRIMARY KEY (userid, chatid)
    );
    """,
    """
    CREATE MATERIALIZED VIEW IF NOT EXISTS chathistory_by_visible AS
    SELECT userid, chatid, chathistoryjson, chattitle, createdon, nettokenconsumption
    FROM chathistory
    WHERE visible IS NOT NULL AND userid IS NOT NULL AND chatid IS NOT NULL
    PRIMARY KEY (visible, userid, chatid);
    """,
    """
    CREATE TABLE IF NOT EXISTS availablemodels (
        deploymentid uuid,
        isactive boolean,
        apikey text,
        deploymentname text,
        endpoint text,
        modelname text,
        modeltype text,
        modelversion text,
        provider text,
        PRIMARY KEY ((deploymentid), isactive)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS usersubscriptions (
        userid uuid,
        modelid uuid,
        PRIMARY KEY (userid, modelid)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS users (
        email text,
        partner text,
        userid uuid,
        firstname text,
        lastname text,
        role text,
        PRIMARY KEY ((email, partner), userid)
    );
    """
]

for query in table_queries:
    session.execute(query)

print("Keyspace and tables created successfully.")
rows = session.execute(f"SELECT table_name FROM system_schema.tables WHERE keyspace_name='{KEYSPACE}';")
for row in rows:
    print("Table:", row.table_name)


In [137]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage, SystemMessage, FunctionMessage, trim_messages
from pydantic import BaseModel, Field
from copy import deepcopy

from typing import Sequence

class InMemoryHistory(BaseChatMessageHistory, BaseModel):
    messages: list[BaseMessage] = Field(default_factory=list)

    def add_messages(self, messages: Sequence[BaseMessage]) -> None:
        self.messages.extend(messages)

    def clear(self) -> None:
        self.messages = []
    
    def edit_message_at_index(self, index: int, new_message: BaseMessage) -> None:
        if 0 <= index < len(self.messages):
            self.messages[index] = new_message
            # Truncate all messages after the edited one
            self.messages = self.messages[:index + 1]
        else:
            raise IndexError("Message index out of range.")

store = {}
branch = "main"

# def get_chat_history_by_branch(branch: str) -> BaseChatMessageHistory:
#     if branch not in store:
#         store[branch] = InMemoryHistory()
#     return store[branch]
def get_chat_history_by_branch(branch: str) -> BaseChatMessageHistory:
    if branch not in store:
        store[branch] = InMemoryHistory()
    return store[branch]

def append_message_to_branch(message: BaseMessage, branch: str) -> None:
    if branch not in store:
        store[branch] = InMemoryHistory()
    store[branch].messages.append(message)

def create_branch_from(parent_branch: str, new_branch: str, edit_index: int, new_message: BaseMessage):
    parent_history = store.get(parent_branch)
    if not parent_history:
        raise ValueError(f"Parent branch '{parent_branch}' does not exist.")

    # Clone messages up to the edit point
    new_history = InMemoryHistory(messages=deepcopy(parent_history.messages[:edit_index]))
    # Add the new edited message
    new_history.add_messages([new_message])
    # Store the new branch
    store[new_branch] = new_history


In [144]:

parser = StrOutputParser()
llm = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    temperature=0.01,
    stream_usage=True
)

In [ ]:
async def process_input(input):
    chat_history = get_chat_history_by_branch(branch).messages[-4:]
    print(chat_history)
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a strict teacher who gives concise answers in 1 word"),
        *chat_history,
        ("human", "{input}"),
    ])
    chain = prompt | llm
    chain_with_history = RunnableWithMessageHistory(
        chain,
        get_session_history=get_chat_history_by_branch,
        input_messages_key="input",
        history_factory_config=[
            ConfigurableFieldSpec(
                id="branch",
                annotation=str,
                name="Chat Branch Name",
                description="Unique name for the chat branch",
                default="",
                is_shared=True,
            ),
        ],
    )
    # ai_message =""
    async for chunk in chain_with_history.astream({"input": input},
                                                  config={"configurable": {"branch": branch}}):
        print(chunk.content, end="", flush=True)
        # ai_message += chunk
    # InMemoryHistory().add_messages([HumanMessage(content=input), AIMessage(content=ai_message)])


In [119]:
branch="part"

In [146]:
await process_input(HumanMessage(content="op"))

[HumanMessage(content='what is earth', additional_kwargs={}, response_metadata={}), AIMessage(content='Planet.', additional_kwargs={}, response_metadata={}), HumanMessage(content='what is earth', additional_kwargs={}, response_metadata={}), AIMessageChunk(content='Planet.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0'}, id='run--93093a8f-bfa9-478c-aa94-c5d009c5205a')]
content='' additional_kwargs={} response_metadata={} id='run--0f30f0a9-380e-4016-85aa-7cfeb1f14573'content='' additional_kwargs={} response_metadata={} id='run--0f30f0a9-380e-4016-85aa-7cfeb1f14573'content='Clar' additional_kwargs={} response_metadata={} id='run--0f30f0a9-380e-4016-85aa-7cfeb1f14573'content='ify' additional_kwargs={} response_metadata={} id='run--0f30f0a9-380e-4016-85aa-7cfeb1f14573'content='.' additional_kwargs={} response_metadata={} id='run--0f30f0a9-380e-4016-85aa-7cfeb1f14573'content='' additional_kwargs={} 

In [147]:
print(store)

{'main': InMemoryHistory(messages=[HumanMessage(content='are you teacher?', additional_kwargs={}, response_metadata={}), AIMessage(content='Yes.', additional_kwargs={}, response_metadata={}), HumanMessage(content='what is earth', additional_kwargs={}, response_metadata={}), AIMessage(content='Planet.', additional_kwargs={}, response_metadata={}), HumanMessage(content='what is earth', additional_kwargs={}, response_metadata={}), AIMessageChunk(content='Planet.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0'}, id='run--93093a8f-bfa9-478c-aa94-c5d009c5205a'), HumanMessage(content='op', additional_kwargs={}, response_metadata={}), AIMessageChunk(content='Clarify.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_ee1d74bde0'}, id='run--0f30f0a9-380e-4016-85aa-7cfeb1f14573', usage_metadata={'input_tokens': 215, 'output_tok

In [92]:
from langchain_core.messages import HumanMessage, AIMessage

# Create a new branch 'feature' by editing message at index 2 ('c') to 'g'
create_branch_from(
    parent_branch='main',
    new_branch='b',
    edit_index=0,
    new_message=HumanMessage(content="check")
)

# Add a new message 'j' to the 'feature' branch
store['b'].add_messages([AIMessage(content="j")])
